# Part 2: Building a Research Assistant Agent with Tools
**Duration:** ~30 minutes

## Learning Objectives
- Build an agent with **multiple tools** using Google ADK
- Implement four real tools: web search, calculator, database query, file reader
- See how the agent **selects the right tool** for each task
- Build a working "Research Assistant" end-to-end

## Prerequisites
- Google API key
- Completed Part 1 (Chatbot to Agent)

In [ ]:
print("Installing dependencies...\n")
%pip install --quiet google-genai google-adk
print("\nAll dependencies installed!")

In [ ]:
import os

api_key = input("Paste your Google API key: ").strip()
if api_key:
    os.environ["GOOGLE_API_KEY"] = api_key
    print("API key configured successfully")
else:
    print("ERROR: API key is required for this notebook")

## The Four Tools We'll Build

An agent is only as capable as its tools. We'll equip our Research Assistant with:

| Tool | Purpose | Example Use |
|------|---------|------------|
| `google_search` | Search the web for information | "Latest AI news" |
| `calculator` | Evaluate math expressions | "What is 15% of 2847?" |
| `query_database` | Query a SQLite database | "How many students scored above 90?" |
| `read_file` | Read contents of text files | "Summarize the contents of report.txt" |

The LLM will automatically decide **which tool to use** based on the question.

In [ ]:
import sqlite3
import os

# --- Create a sample SQLite database ---
DB_PATH = "sample_data.db"

conn = sqlite3.connect(DB_PATH)
cursor = conn.cursor()

# Create students table
cursor.execute("DROP TABLE IF EXISTS students")
cursor.execute("""
    CREATE TABLE students (
        id INTEGER PRIMARY KEY,
        name TEXT,
        major TEXT,
        gpa REAL,
        graduation_year INTEGER
    )
""")

students = [
    (1, "Alice Johnson", "Computer Science", 3.8, 2025),
    (2, "Bob Smith", "Data Science", 3.5, 2025),
    (3, "Carol Williams", "AI & ML", 3.9, 2026),
    (4, "David Brown", "Computer Science", 3.2, 2026),
    (5, "Eve Davis", "Data Science", 3.7, 2025),
    (6, "Frank Miller", "AI & ML", 3.95, 2026),
    (7, "Grace Wilson", "Computer Science", 3.6, 2025),
    (8, "Henry Taylor", "Data Science", 2.9, 2026),
    (9, "Iris Anderson", "AI & ML", 3.85, 2025),
    (10, "Jack Thomas", "Computer Science", 3.1, 2026),
]

cursor.executemany("INSERT INTO students VALUES (?, ?, ?, ?, ?)", students)
conn.commit()
conn.close()

print(f"Created SQLite database: {DB_PATH}")
print(f"  Table: students (10 records)")

# --- Create sample text files ---
os.makedirs("sample_files", exist_ok=True)

with open("sample_files/course_overview.txt", "w") as f:
    f.write("""Course: Introduction to Generative AI and Agentic Systems
Semester: Spring 2025

Course Description:
This course covers the fundamentals of generative AI, from prompt engineering
to building autonomous agents. Students will learn about LLMs, tool use,
the ReAct pattern, and how to build production-ready AI agents.

Grading:
- Assignments: 40%
- Midterm Project: 25%
- Final Project: 35%

Topics:
1. Prompt Engineering Fundamentals
2. From Chatbot to Agent
3. Building Agents with Tools (Google ADK)
4. The ReAct Pattern
5. Guardrails and Safety
6. Multi-Agent Systems
7. Evaluation and Testing
8. Production Deployment
""")

with open("sample_files/research_notes.txt", "w") as f:
    f.write("""Research Notes: State of AI Agents (2025)

Key Findings:
- 73% of enterprise companies are exploring AI agent deployment
- Tool-augmented LLMs outperform base models by 40% on complex tasks
- ReAct pattern improves multi-step reasoning accuracy by 25%
- Average agent handles 4-6 tools effectively
- Main challenges: hallucination control, cost management, latency

Popular Frameworks:
- Google ADK: Production-grade, integrated with Google Cloud
- LangChain/LangGraph: Flexible, large ecosystem
- CrewAI: Multi-agent orchestration
- AutoGen: Microsoft's agent framework

Recommendation: Start with single-agent, tool-augmented systems before
moving to multi-agent architectures.
""")

print(f"Created sample files in: sample_files/")
print(f"  - course_overview.txt")
print(f"  - research_notes.txt")

## Step 1: Define the Tools

Each tool is a Python function with a **clear docstring** — the LLM reads the docstring
to understand when and how to use each tool.

In [ ]:
import sqlite3
from datetime import datetime

def google_search(query: str) -> dict:
    """Search the web using Google Search and return relevant results.
    
    Use this tool when you need to find current information, news,
    facts, or any information that might not be in your training data.
    
    Args:
        query: The search query string
    
    Returns:
        A dictionary containing search results with titles and snippets
    """
    # Note: In production, you'd use the actual Google Search API.
    # For this demo, we return simulated but realistic results.
    simulated_results = {
        "query": query,
        "results": [
            {
                "title": f"Top result for: {query}",
                "snippet": f"Comprehensive information about {query}. This covers the latest developments, key facts, and expert analysis on the topic.",
                "source": "web_search"
            },
            {
                "title": f"Recent developments in {query}",
                "snippet": f"New findings and updates related to {query} as of {datetime.now().strftime('%B %Y')}.",
                "source": "web_search"
            }
        ],
        "total_results": 2,
        "searched_at": datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    }
    return simulated_results


def calculator(expression: str) -> dict:
    """Evaluate a mathematical expression and return the precise result.
    
    Use this tool for any mathematical calculations, arithmetic,
    percentages, or numeric computations.
    
    Args:
        expression: A mathematical expression to evaluate (e.g., '15 * 2847 / 100')
    
    Returns:
        A dictionary with the expression and its result
    """
    import math
    
    safe_dict = {
        "__builtins__": {},
        "abs": abs, "round": round, "min": min, "max": max,
        "sum": sum, "pow": pow, "len": len,
        "sqrt": math.sqrt, "pi": math.pi, "e": math.e,
        "log": math.log, "log10": math.log10,
        "sin": math.sin, "cos": math.cos, "tan": math.tan,
    }
    
    try:
        result = eval(expression, safe_dict)
        return {
            "expression": expression,
            "result": str(result),
            "status": "success"
        }
    except Exception as e:
        return {
            "expression": expression,
            "error": str(e),
            "status": "error"
        }


def query_database(sql_query: str) -> dict:
    """Execute a SQL query against the student database and return results.
    
    Use this tool when you need to look up student records, compute
    statistics about students, or answer questions about academic data.
    
    The database has one table called 'students' with columns:
    - id (INTEGER): Student ID
    - name (TEXT): Full name
    - major (TEXT): Field of study (Computer Science, Data Science, AI & ML)
    - gpa (REAL): Grade point average (0.0 to 4.0)
    - graduation_year (INTEGER): Expected graduation year
    
    Args:
        sql_query: A SQL SELECT query to execute
    
    Returns:
        A dictionary with column names and row data
    """
    try:
        conn = sqlite3.connect("sample_data.db")
        cursor = conn.cursor()
        
        if not sql_query.strip().upper().startswith("SELECT"):
            return {"error": "Only SELECT queries are allowed", "status": "error"}
        
        cursor.execute(sql_query)
        columns = [desc[0] for desc in cursor.description]
        rows = cursor.fetchall()
        conn.close()
        
        return {
            "query": sql_query,
            "columns": columns,
            "rows": [dict(zip(columns, row)) for row in rows],
            "row_count": len(rows),
            "status": "success"
        }
    except Exception as e:
        return {"query": sql_query, "error": str(e), "status": "error"}


def read_file(file_path: str) -> dict:
    """Read the contents of a text file and return it.
    
    Use this tool when you need to read, summarize, or analyze
    the contents of a text file.
    
    Args:
        file_path: Path to the file to read (relative to current directory)
    
    Returns:
        A dictionary with the file path and its contents
    """
    try:
        with open(file_path, "r") as f:
            contents = f.read()
        return {
            "file_path": file_path,
            "contents": contents,
            "char_count": len(contents),
            "line_count": contents.count("\n") + 1,
            "status": "success"
        }
    except FileNotFoundError:
        return {"file_path": file_path, "error": "File not found", "status": "error"}
    except Exception as e:
        return {"file_path": file_path, "error": str(e), "status": "error"}


print("4 tools defined:")
print("  1. google_search(query)      - Search the web")
print("  2. calculator(expression)    - Math calculations")
print("  3. query_database(sql_query) - Query SQLite database")
print("  4. read_file(file_path)      - Read text files")

## Step 2: Create the Research Assistant Agent

Using Google ADK, we wire the tools to an LLM agent. The agent's **instruction**
tells it how to behave, and the **tools** list tells it what capabilities it has.

In [ ]:
from google.adk.agents import Agent as LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.genai import types

# Create the Research Assistant agent
research_agent = LlmAgent(
    name="research_assistant",
    model="gemini-2.5-flash",
    description="A research assistant that can search the web, do calculations, query databases, and read files.",
    instruction="""You are a helpful Research Assistant. You have access to these tools:

1. google_search - Search the web for current information
2. calculator - Perform mathematical calculations
3. query_database - Query the student database (table: students with columns: id, name, major, gpa, graduation_year)
4. read_file - Read text files

When answering questions:
- Use the appropriate tool for the task
- Always explain what you found
- Cite your sources (which tool provided the data)
- If a question requires multiple tools, use them in sequence
- For database queries, write proper SQL
- Be concise but thorough
""",
    tools=[google_search, calculator, query_database, read_file],
)

print("Research Assistant agent created!")
print(f"  Name: {research_agent.name}")
print(f"  Model: {research_agent.model}")
print(f"  Tools: google_search, calculator, query_database, read_file")

In [ ]:
APP_NAME = "research_assistant_app"
USER_ID = "student"
SESSION_ID = "demo_session"

session_service = InMemorySessionService()
runner = Runner(app_name=APP_NAME, agent=research_agent, session_service=session_service)

await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID
)

def ask_research_assistant(question: str, session_id: str = SESSION_ID):
    """Send a question to the research assistant and print the response."""
    print(f"\nQuestion: {question}")
    print("-" * 50)
    
    content = types.Content(
        role="user",
        parts=[types.Part(text=question)]
    )
    
    response_text = ""
    for event in runner.run(
        user_id=USER_ID,
        session_id=session_id,
        new_message=content
    ):
        if hasattr(event, "content") and event.content:
            if hasattr(event.content, "parts"):
                for part in event.content.parts:
                    if hasattr(part, "text") and part.text:
                        response_text += part.text
    
    print(f"\nAnswer:\n{response_text}")
    return response_text

print("Runner and session ready!")

## Step 3: Test Each Tool

Let's verify each tool works by asking questions that target specific tools.
Watch which tool the agent selects for each question.

In [ ]:
print("=" * 60)
print("TEST 1: Web Search Tool")
print("=" * 60)

ask_research_assistant("Search the web for the latest trends in generative AI in 2025")

In [ ]:
print("=" * 60)
print("TEST 2: Calculator Tool")
print("=" * 60)

ask_research_assistant("If a class has 45 students and 78% passed the exam, how many students passed? Also what is the square root of 2025?")

In [ ]:
print("=" * 60)
print("TEST 3: Database Query Tool")
print("=" * 60)

ask_research_assistant("Query the student database to find all students majoring in AI & ML with a GPA above 3.8")

In [ ]:
print("=" * 60)
print("TEST 4: File Reader Tool")
print("=" * 60)

ask_research_assistant("Read the file at sample_files/course_overview.txt and summarize the course topics and grading policy")

## Step 4: Multi-Tool Questions

The real power of an agent is combining multiple tools to answer complex questions.

In [ ]:
print("=" * 60)
print("MULTI-TOOL: Combining Tools for Complex Questions")
print("=" * 60)

# This question might use database + calculator
ask_research_assistant(
    "What is the average GPA of Computer Science students in the database? "
    "How does it compare to the overall average GPA?"
)

In [ ]:
# This question might use file reader + search
ask_research_assistant(
    "Read the research notes file at sample_files/research_notes.txt. "
    "Based on the findings, what percentage improvement does the ReAct pattern give "
    "for multi-step reasoning?"
)

## Step 5: Try It Yourself!

Ask the Research Assistant anything. Try questions that might require:
- Web search ("What is...")
- Calculations ("Calculate...")
- Database queries ("How many students...")
- File reading ("Read the file...")
- Combinations of the above!

In [ ]:
print("=" * 60)
print("INTERACTIVE: Ask the Research Assistant")
print("Type 'quit' to exit")
print("=" * 60)

# Create a fresh session for interactive use
INTERACTIVE_SESSION = "interactive_session"
await session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID, session_id=INTERACTIVE_SESSION
)

while True:
    question = input("\nYour question: ").strip()
    if question.lower() in ("quit", "exit", "q"):
        print("Session ended.")
        break
    if not question:
        continue
    ask_research_assistant(question, session_id=INTERACTIVE_SESSION)

## Summary

### What We Built
A **Research Assistant** agent with 4 tools using Google ADK.

### Key Concepts
1. **Tool Definition**: Each tool is a Python function with a clear docstring
2. **Agent Creation**: Wire tools to an LLM via `LlmAgent(tools=[...])`
3. **Tool Selection**: The LLM reads docstrings and decides which tool to use
4. **Multi-Tool Usage**: Complex questions can trigger multiple tool calls
5. **ADK Runtime**: `Runner` + `SessionService` manages the execution loop

### Architecture
```
User Question -> ADK Runner -> LLM (Gemini)
                                   |
                            Tool Decision
                           /    \    \     \
                     search  calc  db   file
                           \    /    /     /
                         Tool Results
                                   |
                              LLM Summary
                                   |
                              Final Answer
```

**Next:** Part 3 - The ReAct Pattern (Reasoning + Acting)

In [ ]:
# Clean up sample data
import os

if os.path.exists("sample_data.db"):
    os.remove("sample_data.db")
if os.path.exists("sample_files/course_overview.txt"):
    os.remove("sample_files/course_overview.txt")
if os.path.exists("sample_files/research_notes.txt"):
    os.remove("sample_files/research_notes.txt")
if os.path.exists("sample_files"):
    os.rmdir("sample_files")

print("Sample data cleaned up.")